In [1]:
%load_ext jupyter_black
from rich import print, print_json

# Case Study of OOP (Payment Service Integration with Payment Service Provider)

**Problem Description:** 

Design a high level OOP design of a system that handles payments from such as bKash, nagad & also payment service provider like Easten Bank Ltd. Payment gateway and CityBank.

**Functional Requirements:**

- Pay-in-flow: Payment system receives money from customer and make the premium sattelement.

**Non Functional Requirements:**

- Reliability: handle failed payments
- proper logging at each steps
- Follow Clean Architechture
- Modular design

**High-Level Premium Pay Flow**

![psp_diagram](assets/use_case_diagram_psp.png "High-Level Premium Pay Flow")

**Class Entities**

- Payment Service
    - The payment service accepts payment events from users and coordinates the payment process. The first thing is usually does is check of business rules for a single policy object payments.Also it should have a functionlity for generate a unique order id for each payment execution 
- Payment Executor
    - The payment executor executes a single payment order via a payment service provider(PSP).
- Ledger
    - The ledger keeps a financial record of the payment transaction. 
- Insurer
    - The Insurer is the policy object information from the system, where a details of the policy is found.

In [2]:
import string
import random
from abc import ABC, abstractmethod
from datetime import date
from enum import Enum
from domain.plil import Insurer, Policy


class ServiceProvider(str, Enum):
    MFS = "MFS"
    PG = "PAYMENT_GATEWAY"
    AB = "AGENT_BANKING"


class IPaymentService(ABC):
    """Abstract class to blueprint the inherited classes should have"""

    @abstractmethod
    def create_orderid(self) -> None:
        raise NotImplementedError

    @abstractmethod
    def order_id(self) -> str:
        raise NotImplementedError


class PaymentService(IPaymentService):
    def __init__(self, insurer_obj: Insurer) -> None:
        self._orderid = None
        self._insurer = insurer_obj

    def is_policy_lapsed(self) -> bool:
        return self._insurer.lapsed

    def is_premium_paid(self) -> bool:
        return self._insurer.due_amount() <= 0

    def create_orderid(self, office_code: str, orderid_length: int = 4) -> None:
        current_date = date.today().strftime("%y%m%d")

        # Generate random letters without digits
        random_letters = "".join(
            random.choices(string.ascii_uppercase, k=orderid_length - 2)
        )

        # Generate two random digits
        random_digits = "".join(random.choices(string.digits, k=2))

        # Insert the two random digits at random positions within the random_letters
        random_position = random.randint(0, len(random_letters))
        random_letters = (
            random_letters[:random_position]
            + random_digits
            + random_letters[random_position:]
        )

        self._orderid = f"{office_code}{current_date}{random_letters}"

    @property
    def order_id(self) -> str | None:
        return self._orderid


class MFSPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self._provider = ServiceProvider.MFS


class ABPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self._provider = ServiceProvider.AB


class PGPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self._provider = ServiceProvider.PG


class IPaymentExecutor(ABC):
    @abstractmethod
    def execute(self) -> None:
        raise NotImplementedError


class PaymentExecutor(IPaymentExecutor):
    def __init__(self, payment_service: PaymentService) -> None:
        self.payment_service = payment_service

    def execute(self) -> None:
        print(
            f"Executing Payment from Payment Executor for orderid={self.payment_service.order_id}"
        )

In [3]:
policy_obj = Policy(
    policy_number="0134050541",
    total_premium=2700,
    premium_no=7,
    paymode="5",
    next_bill_date=date(2023, 7, 13),
    insurer_name="K M JIAUL ISLAM JIBON",
    commencement_date=date(2022, 7, 10),
    birthday=date(1994, 3, 10),
    maturity_date=date(2032, 7, 10),
    plan_code="212-10",
    sum_assured=3_00_0000,
    project_code="01",
    organization_setup="org_setup_string",
    total_paid=7500.00,
    project_category="i",
)
insurer = Insurer(policy_obj)
ps = PaymentService(insurer)
pe = PaymentExecutor(ps)

ps.create_orderid("0134")

if ps.order_id:
    ledger = insurer.ledger(order_id=ps.order_id)
    print(ledger)

Ledger(
    policy_number='0134050541',
    insurer_name='K M JIAUL ISLAM JIBON',
    plan_code='212-10',
    paymode='5',
    premium_no=7,
    sum_assured=3000000.0,
    project_category='i',
    organization_setup='org_setup_string',
    commencement_date=datetime.date(2022, 7, 10),
    total_paid=7500.0,
    next_bill_date=datetime.date(2023, 7, 13),
    late_fee=0.0,
    suspense_amount=0.0,
    policy_status=None,
    phone_number=None,
    total_premium=2700.0,
    total_paid_amount=0.0,
    order_id='0134230724Q07X',
    receipt_id=None,
    user_id=None,
    bank_code=None,
    project_code='01',
    next_premium_no=0,
    next_total_payed_amount=0.0,
    next_suspense_amount=0.0,
    receipt_date=None,
    count_of_installment=0,
    total_payable_amount=0.0,
    discount_amount=0.0,
    net_received_amount=0.0,
    payment_status=None
)